In [4]:
import pandas as pd
import numpy as np

In [5]:
PATH = 'D:\\Run Analysis\\blob_storage\\truck_exp_other\\original_ds\\20211024_motorVehicle.xlsx'
COMPLETEPATH = 'D:\\Run Analysis\\blob_storage\\truck_exp_other\\transformed_og_ds\\40th_2021.csv'   
df = pd.read_excel(PATH)
df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y")
df[df.Date.between('20211006', '20211012')].to_csv(COMPLETEPATH,index=False)

In [6]:
WEEK = "40th_2021"

PATH_TRANSFORMED_MVEXP = f"D:\\Run Analysis\\blob_storage\\truck_exp_other\\transformed_og_ds\\{WEEK}.csv"

PATH_ROSTER = f"D:\\Run Analysis\\blob_storage\\roster\\{WEEK}.xlsx" 

mvexp_df = pd.read_csv(PATH_TRANSFORMED_MVEXP)

roster_df = pd.read_excel(PATH_ROSTER)


In [7]:
mvexp_df_trucks_arr = mvexp_df['Job No.'].unique()

In [8]:
pri_trucks = roster_df.dropna(subset=['Primary_truck']).Primary_truck.str.replace(' ','').unique()
sec_trucks = roster_df.dropna(subset=['Alternative_Truck']).Alternative_Truck.str.replace(' ','').unique()
# pri_trucks
arr = np.append(pri_trucks, sec_trucks)
uniqArr = list(set(arr))
ros_trucks = np.array(uniqArr)


In [9]:
np.setdiff1d(ros_trucks,mvexp_df_trucks_arr  ,assume_unique=True)

array(['XN72FD', 'XN75OS', 'CG94KY', 'CH32FV', 'XN10HU', 'CJ65PS',
       'XO62AB', 'XN81VB', 'XN76VB', 'CL98LU', 'XN19IN', 'XN71FD',
       'XN65IJ', 'XN70OD', 'XN25GD', 'XN62UD', 'XN44QZ', 'CQ64LO'],
      dtype='<U6')

In [10]:
np.setdiff1d(mvexp_df_trucks_arr,ros_trucks  ,assume_unique=True)

array(['New Front Lift', 'New Hook Lift', 'FLEET', 'XN13KR'], dtype=object)

In [11]:
PATH = "D:\\Run Analysis\\blob_storage\\roster\\32th_2021.xlsx"
roster_df = pd.read_excel(PATH)


def clean_roster_df(df):
    df.Primary_truck = df.Primary_truck.str.replace(" ", "")
    df.Alternative_Truck = df.Alternative_Truck.str.replace(" ", "")
    df = df.fillna(0)
    return df

roster_df = clean_roster_df(roster_df)


pri_truck_runs_count = roster_df.Primary_truck.value_counts()
sec_truck_runs_count = roster_df.Alternative_Truck.value_counts()

# =============================================================================================
# Counting Trucks runs
total_truck_runs_count = (pd
                            .concat([pri_truck_runs_count, sec_truck_runs_count])
                            .reset_index()
                            .groupby('index')
                            .sum().iloc[1:]
                            .reset_index()
                            .rename(columns={"index" : "number_plate", 0: "runs"})
                            )
df_99 = pd.merge(mvexp_df, total_truck_runs_count, how="left", left_on='Job No.', right_on='number_plate')


In [12]:
df_99['per_run_cost']= df_99.Debit / df_99.runs

df_99[['Date','Acc_name', 'Job No.', 'Debit','per_run_cost']]

,Date,Acc_name,Job No.,Debit,per_run_cost
0,2021-10-07,MV R&M - Cab&Chassic,BY46TO,99.28,19.856000
1,2021-10-08,MV R&M - Cab&Chassic,XN80YD,69.05,5.311538
2,2021-10-11,MV R&M - Cab&Chassic,New Front Lift,84.00,NaN
3,2021-10-11,MV R&M - Cab&Chassic,XPL658,150.00,25.000000
4,2021-10-11,MV R&M - Cab&Chassic,CF48ED,150.00,75.000000
5,2021-10-11,MV R&M - Cab&Chassic,New Hook Lift,150.00,NaN
6,2021-10-11,MV R&M - Cab&Chassic,BL82AG,150.00,37.500000
7,2021-10-11,MV R&M - Cab&Chassic,XN72MK,150.00,30.000000
8,2021-10-12,MV R&M - Cab&Chassic,CF71JM,360.41,72.082000
9,2021-10-08,MV R&M - Compactor/Body,CF71JM,284.02,56.804000


In [43]:
total_truck_runs_count

,number_plate,runs
0,BL82AG,4
1,BY46TO,5
2,CF48ED,2
3,CF71JM,5
4,CH32FV,2
5,CL02TB,4
6,CL98LU,5
7,CN41XN,5
8,CQ64LO,9
9,X062AB,1


In [13]:
roster_df

,Date,Run_type,Primary_employeeID,Secondary_employeeID,Primary_driver_Name,Secondary_Driver_Name,Primary_route,Satellite_Route_1,Satellite_Route_2,Primary_truck,Alternative_Truck,Subcontracted_From/Special_Client,Note
0,2021-08-11,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
1,2021-08-12,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
2,2021-08-13,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
3,2021-08-16,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,0,0.0
4,2021-08-17,NIGHT RUNS,swaklado,0,ADONIS AKL,0,RLE,0,0,CL98LU,0,UNIVERSITY,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,2021-08-12,DAY RUNS,irdeaz,0,ZAC DEAN,0,RL7,0,0,XN44QZ,0,0,0.0
129,2021-08-13,DAY RUNS,irdeaz,0,ZAC DEAN,0,RL7,0,0,XN44QZ,0,0,0.0
130,2021-08-14,DAY RUNS,irdeaz,0,ZAC DEAN,0,RL7,0,0,XN44QZ,0,0,0.0
131,2021-08-17,DAY RUNS,irdeaz,0,ZAC DEAN,0,RL7,0,0,XN44QZ,0,0,0.0
